In [ ]:
from PyQt5.QtCore import QDate, Qt


In [ ]:
QDate.currentDate().string()

In [ ]:
import os
import pandas as pd
if os.path.isfile(r'E:\source\pygame\Nado Game\pyqt5\자재관리\입고대장.xlsx'):
    LE =  [
        'E:/source/pygame/Nado Game/pyqt5/자재관리/입고대장.xlsx',
        'E:/source/pygame/Nado Game/pyqt5/자재관리/사용대장.xlsx',
        'E:/source/pygame/Nado Game/pyqt5/자재관리/동호대장.xlsx',
        ]
else: 
    LE =  [
        'C:/source/pygame/Nado Game/pyqt5/자재관리/입고대장.xlsx',
        'C:/source/pygame/Nado Game/pyqt5/자재관리/사용대장.xlsx',
        'C:/source/pygame/Nado Game/pyqt5/자재관리/동호대장.xlsx',
        ]
def out_df():
    file = LE[1] #r'C:\source\pygame\Nado Game\pyqt5\자재관리\사용대장.xlsx'
    with pd.ExcelFile(file) as f:
        df = pd.read_excel(f,skiprows=0)
    return df
def in_df():
    file = LE[0] #r'C:\source\pygame\Nado Game\pyqt5\자재관리\입고대장.xlsx'
    with pd.ExcelFile(file) as f:
        df = pd.read_excel(f,skiprows=0)
    return df
def df_status_selection(df):
    from_date = '2022-02-21'
    to_date = '2022-05-26'
    items = 'LED 전구'
    spec = '11w'
    date_to_con = df['일자'] <= to_date
    date_from_con = df['일자'] >= from_date
    i_con = df['품명'] == items
    s_con = df['규격'] == spec
    if items == 'All':
        df_sel = df[(date_from_con & date_to_con)]
        return df_sel
    else:
        df_sel = df[(date_from_con & date_to_con & i_con & s_con)]
        return df_sel
def out_status_selection(df):
    from_date = '2022-02-26'
    to_date = '2022-05-26'
    items = 'LED 전구'
    spec = '11w'
    d_to_con = df['사용일'] <=to_date
    d_from_con = df['사용일'] >=from_date
    i_con = df['품명'] == items
    s_con = df['규격'] == spec
    if items == 'All':
        df_sel = df[(d_to_con & d_from_con)]
        return df_sel
    else:
        df_sel = df[(d_to_con & d_from_con & i_con & s_con)]
        return df_sel


def detailed_in_out_view():
    dfI = in_df()
    dfI['입출'] ='입고'
    dfI.rename(columns = {'입고일':'일자'}, inplace=True)
    dfIn = df_status_selection(dfI)
    #print(dfIn)
    dfO = out_df()
    
    dfO['입출'] = '사용'
    dfO.rename(columns = {'사용일':'일자'}, inplace=True)
    dfOut = df_status_selection(dfO)
    #print(dfIn)
    items_in = set(dfIn['품명'].unique())
    specs_in = set(dfIn['규격'].unique())
    items_out = set(dfOut['품명'].unique())
    specs_out = set(dfOut['규격'].unique())
    items = list(items_in.union(items_out))
    specs = list(specs_in.union(specs_out))
    temp_list = []
    for item in items:
        #print(item)
        for spec in specs:
            #print(spec)
            #print(dfIn)
            dfIn_sel = dfIn[((dfIn['품명'] == item) & (dfIn['규격'] == spec))]
            #print('dfIn_sel',dfIn_sel)
            dfOut_sel = dfOut[((dfOut['품명'] == item) & (dfOut['규격'] ==  spec))]
            df_on_stock = pd.merge(dfIn_sel, dfOut_sel, how = 'outer', on = ['입출','일자','품명','규격'])
            df_on_stock.fillna(0, inplace=True)
            #print(df_on_stock)
            df_on_stock.sort_values(by= '일자', inplace=True)
            df_total = df_on_stock[['일자','품명','규격','입출','입고수량','구입금액','단가','구분','동','호','사용수량','품목누계']]
            df_total.reset_index(drop=True, inplace=True)
            df_total['sum'] = df_total['입고수량'] - df_total['사용수량']
            df_temp = df_total[['sum']].copy()
            df_temp_1  = df_temp.cumsum()
            df_total['sum'] = df_temp_1['sum']
            df_total[['입고수량','구입금액','단가','동','호','사용수량','품목누계','sum']].astype('int')
            #print(df_total)
            if df_total.empty:
                pass
            else:
                temp_list.append(df_total)
    
    return temp_list
'''
Total = detailed_in_out_view()
df_con = Total[0]
for part in Total[1:]:
    df_con = pd.concat([df_con, part])
    df_con.reset_index(drop=True, inplace=True)
df_con[['입고수량','구입금액','단가','동','호','사용수량','품목누계','sum']] = df_con[['입고수량','구입금액','단가','동','호','사용수량','품목누계','sum']].astype('int')
df_con[['입고수량','구입금액','단가','동','호','사용수량','품목누계','sum']] = df_con[['입고수량','구입금액','단가','동','호','사용수량','품목누계','sum']].astype('str')
df_1 =df_con.set_index(['품명'])

df_1.loc['LED 전구', '입고수량']
'''

In [ ]:
def in_stock_view():
    list = ['LED 전구', '11w']
    on_stock_qty = items_spec_onstock(list[0], list[1])
    print(on_stock_qty)

def items_spec_onstock(items, spec):
    df_on_stock = on_stock()
    print(df_on_stock)
    print(items, spec)
    df_on_stock.set_index(['품명', '규격'], inplace=True)
    
    try:
        on_stock_qty = df_on_stock.loc[[(items, spec)],'재고'].values.tolist()
        on_stock_qty = on_stock_qty[0]
        #on_stock_qty=int(on_stock_qty)
        return on_stock_qty
    except:
        on_stock_qty = '신규항목임'
        #rint(on_stock_qty)
        return on_stock_qty
    
def on_stock():
    dfIn = in_df()
    df_in = dfIn[['품명','규격', '입고수량']].copy()
    df_in_sum = df_in.groupby(['품명','규격']).sum()

    dfOut = out_df()
    df_out = dfOut[['품명','규격', '사용수량']].copy()
    df_out_sum = df_out.groupby(['품명','규격']).sum()

    df_on_stock = pd.merge(df_in_sum, df_out_sum, how = 'outer', on = ['품명','규격'])
    df_on_stock.fillna(0, inplace=True)
    try:
        df_on_stock['재고'] = df_on_stock['입고수량'] - df_on_stock['사용수량']
        df_on_stock[['입고수량', '사용수량','재고']] = df_on_stock[['입고수량', '사용수량','재고']].astype('str')
        df_m = df_on_stock.reset_index().copy()
        return df_m
    except:
        pass
in_stock_view()

In [ ]:
dfI

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    """여러 데이터프레임 비교가 쉽게 옆쪽으로 표시한다"""
    html_str=''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)
dfo =out_df()
dfi =in_df()
display_side_by_side(dfi,dfo)

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    """여러 데이터프레임 비교가 쉽게 옆쪽으로 표시한다"""
    html_str=''
    for df in args:
        html_str += df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'), raw=True)

In [ ]:
dfo =out_df()
dfi =in_df()
dfiq = dfi.query('(품명 == "LED 전구") and (규격 == "11w")')
dfiqs = dfiq.sort_values(by=['입고일'])
#dfiqs['sum'] = dfiqs.cumsum([['입고수량']])
dfiqs['cumsum'] = dfiqs.groupby((dfiqs[[규격]]).cumsum()).입고수량.cumsum()

In [ ]:
dfo =out_df()
dfoq = dfo.query('(품명 == "LED 전구") and (규격 == "11w")')
dfoqs = dfoq.sort_values(by=['사용일'])
dfoqs

In [ ]:
#df['cumsum'] = df.groupby((df.NUM == 10).cumsum()).ENERGY.cumsum()
#출처: https://note.espriter.net/1339 [espriter:티스토리]
dfo =in_df()
dfo[['구입금액', '단가']].astype('str')
dfo.groupby(['품명', '규격','입고일']).sum(['입고수량']).groupby(level=0).cumsum().reset_index()


In [ ]:
# def cum_in_calc(self,df)
dfo =out_df()
items = dfo['품명'].unique()
#spec = dfo['규격'].unique() 
df_list = []
for i in items:
    dfo_s = dfo[(dfo['품명'] == i)]
    spec = dfo_s['규격'].unique()
    for s in spec:
        
        dfo_sel = dfo[(dfo['품명'] == i) & (dfo['규격'] == s)].copy()# & dfo['규격'] == '11w')]
        dfo_temp = dfo_sel.sort_values(by=['입고일']).copy()
        dfo_copy = dfo_temp[['입고수량']].copy()
        dfo_copy['품목 누계'] = dfo_copy.cumsum()
        dfo_temp['품목 누계']=dfo_copy['품목 누계']
        df_list.append(dfo_temp)
        
df_list[1]
df_con = df_list[0]
for l in df_list[1:]:
    df_con = pd.concat([df_con, l])
df_con
df_con.columns.tolist()
df_con[['입고일', '품명', '규격', '입고수량', '품목 누계', '구입금액', '단가', '구입업체', '비고' ]]
df_con.sort_values(by=['입고일', '품명','규격'])   
#display_side_by_side(dfo_temp, dfo_copy)

In [ ]:
def in_out_status(df):
        items = df['품명'].unique()
        df_lists = []
        for item in items:
            df_specs = df[(df['품명'] == item)]
            specs = df_specs['규격'].unique()
            for spec in specs:
                df_sel = df[(df['품명'] == item) & (df['규격'] == spec)].copy()
                try:
                    df_temp = df_sel.sort_values(by=['입고일']).copy()
                    df_copy = df_temp[['입고수량']].copy()
                except:
                    df_temp = df_sel.sort_values(by=['사용일']).copy()
                    df_copy = df_temp[['사용수량']].copy()
                df_copy['품목누계'] = df_copy.cumsum()
                df_temp['품목누계']=df_copy['품목누계']
                df_lists.append(df_temp)
        df_con = df_lists[0]
        for df_list in df_lists[1:]:
            df_con = pd.concat([df_con, df_list])
        #df_con.columns.tolist()
        try:
            df_con_1= df_con[['입고일', '품명', '규격', '입고수량', '품목누계', '구입금액', '단가', '구입업체', '비고' ]]
            df_con_1.sort_values(by=['입고일', '품명','규격'], inplace=True)
            df_con_1[['입고수량', '구입금액','단가','품목누계']] = df_con[['입고수량', '구입금액','단가','품목누계']].astype('str')
        except:
            df_con_1 = df_con[['사용일', '동','호','구분','품명', '규격', '사용수량', '품목누계', '비고' ]]
            
            df_con_1.sort_values(by=['사용일', '품명','규격'])
            df_con_1[['사용수량', '동','호']] = df_con[['사용수량', '동','호']].astype('str')

        return df_con_1
dfo =out_df()
df_out=in_out_status(dfo)
df_out

In [ ]:
dfo['구분'] = '입고'
dfo_stack=dfo.set_index(['구분','품명','규격','입고일']).stack()
dfo_stack.reset_index()
#fo_stack#['구분'] = '입고'
#dfo_stack.reset_index()


In [ ]:
from PyQt5.QtCore import QDate
qDateOutQueryStart = QDate.currentDate()
qDateOutQueryStart.addMonths(-3)
dateEdit_3.setDate(qDateOutQueryStart)